In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os

In [2]:
start_time = datetime.now()

In [3]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.canada.ca/en/environment-climate-change/services/canadian-environmental-protection-act-registry/substances-list/toxic/schedule-1.html")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value="/html/body/main/div[1]/div/ol/li[2]/a").click()
Register_number = []
for i in range(163):
    try:
        driver.get("https://www.canada.ca/en/environment-climate-change/services/canadian-environmental-protection-act-registry/substances-list/toxic/schedule-1.html")
        driver.implicitly_wait(5)
        driver.find_element(by=By.XPATH, value="/html/body/main/div[1]/div/ol/li[" + str(i) + "]/a").click()
        get_url = driver.current_url
        page = requests.get(get_url)
        soup = BeautifulSoup(page.content, "html.parser")
        for para in soup.find_all("p"):
            string = para.get_text()
            if re.search("[0-9]+-[0-9]+-[0-9]+", string) and  "CAS" in string:
                d = re.findall("[0-9]+-[0-9]+-[0-9]+", string)
                Register_number.extend(d)
        for para in soup.find_all("li"):
            string = para.get_text()
            if re.search("[0-9]+-[0-9]+-[0-9]+", string) and  "CAS" in string:
                d = re.findall("[0-9]+-[0-9]+-[0-9]+", string)
                Register_number.extend(d)
    except:
        pass

In [4]:
Register_number
mylist = list(dict.fromkeys(Register_number))
print(len(mylist))

69


In [16]:
dataset = pd.DataFrame(mylist,columns =['CAS Number'])
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not in list'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.head(2)

,CAS Number,value
0,61788-33-8,NaN
1,1332-21-4,NaN


In [17]:
result['Jurisdiction'] = 'Canada'
result['RegulatoryAgency'] = 'Canada Environmental Protection Agency (CEPA)'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Toxocity'


result['Description'] = 'Toxic'
result.loc[result['value'] == 'Not in list', 'Description'] = 'Not prioritized'

result['HazardClassification'] = 'High'
result.loc[result['value'] == 'Not in list', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'On list of Toxic Substances (Schedule 1)'
result.loc[result['value'] == 'Not in list', 'CauseOfClassification'] = 'Not on Toxic Substances List or under assessment'

result['SourceName'] = 'Health Canada'
result['SourceURL'] = 'https://www.canada.ca/en/environment-climate-change/services/canadian-environmental-protection-act-registry/substances-list/toxic/schedule-1.html'

result = result.drop(['value'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,61788-33-8,Canada,Canada Environmental Protection Agency (CEPA),Health,Toxocity,Toxic,High,On list of Toxic Substances (Schedule 1),Health Canada,https://www.canada.ca/en/environment-climate-c...
1,1332-21-4,Canada,Canada Environmental Protection Agency (CEPA),Health,Toxocity,Toxic,High,On list of Toxic Substances (Schedule 1),Health Canada,https://www.canada.ca/en/environment-climate-c...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [18]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:53:42.633943
